In [13]:
import pandas as pd # 데이터프레임 사용을 위해
import math # IDF 계산을 위해

docs = [
  'new home sales top forecasts',
  'home sales rise in july',
  'incresase in home sales in july',
  'new home sales rise in november'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
#docs를 doc으로 나눈다
#doc을 w로 나눈다
#print(vocab)


vocab.sort() #정렬
#print(vocab)

N = len(docs) # 총 문서의 수

#def tf(t, d):
#    return d.count(t)

def tf(t,d): #"increase" 에서 "in" count되는거 방지
    num=0
    d=d.split(" ")
    for i in d:
        if(i==t):
            num +=1
    return num

def idf(t):
    #vocab = ['forecasts', 'home', 'in', 'incresase', 'july', 'new', 'november', 'rise', 'sales', 'top']      
    #t에 vocab 하나씩 들어온다
    df = 0
    for doc in docs:#한줄씩 삽입
        df += t in doc
    #return log(N/(df + 1))
    return math.log10(N/df)

#밑이 e인 자연 log :log() 
#상용 log : log10()
#밑이 2인 log :log(, 2)


def tfidf(t, d):
    return tf(t,d)* idf(t)

In [14]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    
    result.append([]) #마지막 자리 하나씩 만들기 result[-1]통해 대입
    
    d = docs[i] # d = 문장 순서 (한문장씩)
    for j in range(len(vocab)):
        
        t = vocab[j]  
        #vocab = ['forecasts', 'home', 'in', 'incresase', 'july', 'new', 'november', 'rise', 'sales', 'top']      
        #t에 vocab 하나씩 들어감
        
        result[-1].append(tf(t, d)) #d에서 t계수 세기 ,문장에서 t변수 하나씩 체크
        # a[-1]은 문자열에서와 마찬가지로 리스트 a의 마지막 요솟값

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,forecasts,home,in,incresase,july,new,november,rise,sales,top
0,1,1,0,0,0,1,0,0,1,1
1,0,1,1,0,1,0,0,1,1,0
2,0,1,2,1,1,0,0,0,1,0
3,0,1,1,0,0,1,1,1,1,0


In [15]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    #vocab = ['forecasts', 'home', 'in', 'incresase', 'july', 'new', 'november', 'rise', 'sales', 'top']      
    #t에 vocab 하나씩 들어감
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
forecasts,0.602060
home,0.000000
in,0.124939
incresase,0.602060
july,0.301030
new,0.301030
november,0.602060
rise,0.301030
sales,0.000000
top,0.602060


In [16]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,forecasts,home,in,incresase,july,new,november,rise,sales,top
0,0.60206,0.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.0,0.60206
1,0.00000,0.0,0.124939,0.00000,0.30103,0.00000,0.00000,0.30103,0.0,0.00000
2,0.00000,0.0,0.249877,0.60206,0.30103,0.00000,0.00000,0.00000,0.0,0.00000
3,0.00000,0.0,0.124939,0.00000,0.00000,0.30103,0.60206,0.30103,0.0,0.00000


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'new home sales top forecasts',
  'home sales rise in july',
  'incresase in home sales in july',
  'new home sales rise in November',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.56199026 0.29326983 0.         0.         0.         0.44307958
  0.         0.         0.29326983 0.56199026]
 [0.         0.35220727 0.43080005 0.         0.53212376 0.
  0.         0.53212376 0.35220727 0.        ]
 [0.         0.26784458 0.65522475 0.51326809 0.40466646 0.
  0.         0.         0.26784458 0.        ]
 [0.         0.2919358  0.35707939 0.         0.         0.44106408
  0.55943386 0.44106408 0.2919358  0.        ]]
{'new': 5, 'home': 1, 'sales': 8, 'top': 9, 'forecasts': 0, 'rise': 7, 'in': 2, 'july': 4, 'incresase': 3, 'november': 6}


In [18]:
#코사인 유사도
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([1,1,0,0,0,1,0,0,1,1])
doc2 = np.array([0,1,1,0,1,0,0,1,1,0])
doc3 = np.array([0,1,2,1,1,0,0,0,1,0])
doc4 = np.array([0,1,1,0,0,1,1,1,1,0])

print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc1, doc4))
print(cos_sim(doc2, doc3))
print(cos_sim(doc2, doc4))
print(cos_sim(doc3, doc4))

0.3999999999999999
0.31622776601683794
0.5477225575051661
0.7905694150420948
0.7302967433402214
0.5773502691896258
